In [410]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import pickle
import datetime
import numpy as np
import random
import seaborn as sns

Specifying all the focal brands

In [411]:
focal_brands = ['Sephora',
 'ULTA Beauty',
 'Olive Garden',
 'The Cheesecake Factory',
 'Target',
 'Walmart',
 'Anthropologie',
 "Victoria's Secret"]

Reading the social brands catalog to get visits later for each store

In [412]:
brands_visits = pd.read_csv('../data/revision_visits_revenue_2019.csv')
brands_visits['brand_standard'] = brands_visits['brand'].apply(lambda x: x.strip().lower()) # For comparison with catalog.tsv
brands_visits['date'] = brands_visits['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date())
brands_visits = brands_visits.rename(columns={'brand': 'brand_visitation'})
brands_visits.head()

,date,PLACEKEY,visits_by_day,spend_by_day,brand_visitation,lat,lon,brand_standard
0,2019-06-01,zzw-222@62j-sgj-q2k,5,0.0,Orangetheory Fitness,42.350592,-71.153024,orangetheory fitness
1,2019-06-02,zzw-222@62j-sgj-q2k,1,0.0,Orangetheory Fitness,42.350592,-71.153024,orangetheory fitness
2,2019-06-03,zzw-222@62j-sgj-q2k,6,859.0,Orangetheory Fitness,42.350592,-71.153024,orangetheory fitness
3,2019-06-04,zzw-222@62j-sgj-q2k,6,30.0,Orangetheory Fitness,42.350592,-71.153024,orangetheory fitness
4,2019-06-05,zzw-222@62j-sgj-q2k,8,193.0,Orangetheory Fitness,42.350592,-71.153024,orangetheory fitness


Reading Travel Time

In [413]:
with open('../data/travel_time.pkl', 'rb') as file:
    travel_time_dict = pickle.load(file)
    
travel_time_keys = list(travel_time_dict.keys())
from_keys = [key[0] for key in travel_time_keys]
to_keys = [key[1] for key in travel_time_keys]
time_minutes = list(travel_time_dict.values())
time_minutes = [int(time_inst.split(' ')[0]) for time_inst in time_minutes]

travel_time = pd.DataFrame({'From_PLACEKEY': from_keys, 'To_PLACEKEY': to_keys, 'Time_mins': time_minutes})
travel_time.head()

,From_PLACEKEY,To_PLACEKEY,Time_mins
0,zzw-224@62k-p96-s5z,zzw-223@62k-ns4-pn5,20
1,zzw-224@62k-p96-s5z,zzy-222@62k-pd8-975,20
2,zzw-224@62k-p96-s5z,237-222@62k-p8v-z4v,16
3,zzw-224@62k-p96-s5z,222-222@62k-p8v-2p9,12
4,zzw-224@62k-p96-s5z,229-222@62k-p76-d9z,14


Reading the statistics of the specific focal brand using the results of Part 2

In [823]:
brand = focal_brands[7]
focal_brand_path = os.path.join('../part2_r_statistics', brand)
focal_brand_path

"../part2_r_statistics/Victoria's Secret"

Reading all the neighboring brands results and stats for the selected focal brand

In [824]:
file_list = os.listdir(focal_brand_path)
result_file_list = [file for file in file_list if file.find('_result') != -1]
stat_file_list = [file for file in file_list if file.find('_stat') != -1]

In [825]:
result_df = pd.DataFrame()

for result_file in result_file_list:
    result_file_path = os.path.join(focal_brand_path, result_file)
    tmp_res_df = pd.read_csv(result_file_path, skiprows=1, float_precision="round_trip")
    # Removing all the records for ols and m_olsExp
    if 'filename' in tmp_res_df['filename'].tolist(): #Wrong input
        continue
    result_df = pd.concat([result_df, tmp_res_df], ignore_index=True)

In [826]:
full_result_df = result_df
result_df = result_df[~result_df['type'].isin(['ols', 'm_olsExp'])].reset_index(drop=True)
result_df = result_df.replace('FALSE', False).replace('False', False)

In [827]:
stat_df = pd.DataFrame()

for stat_file in stat_file_list:
    stat_file_path = os.path.join(focal_brand_path, stat_file)
    tmp_stat_df = pd.read_csv(stat_file_path, skiprows=1, float_precision="round_trip")
    # Removing all the records for ols and m_olsExp
    if 'filename' in tmp_stat_df['fileName'].tolist(): #Wrong input
        continue
    stat_df = pd.concat([stat_df, tmp_stat_df], ignore_index=True)

In [828]:
stat_df

,tmp,fileName,z_stat,p_value,z_stat_exp,p_value_exp
0,tmp,Enterprise Rent-A-Car,-0.361435,0.717774,-0.927450,0.353693
1,tmp,Exxon Mobil,0.110637,0.911904,14.089292,0.000000
2,tmp,Cellular Sales,0.722904,0.469739,1.171016,0.241592
3,tmp,lululemon athletica,0.251839,0.801166,-0.245443,0.806113
4,tmp,Dave & Buster's,0.015407,0.987707,0.097575,0.922269
...,...,...,...,...,...,...
378,tmp,Pure Hockey,0.307605,0.758383,-0.451526,0.651611
379,tmp,T,-0.607898,0.543255,-0.449707,0.652922
380,tmp,Converse,0.717293,0.473193,-0.347909,0.727909
381,tmp,Dollar Tree,-0.332118,0.739800,-0.369347,0.711869


In [829]:
sum(result_df.groupby('filename')['type'].nunique() == 1)

0

In [830]:
sum(result_df.groupby('filename')['type'].nunique() == 2)

6

In [831]:
sum(result_df.groupby('filename')['type'].nunique() == 3)

5

In [832]:
sum(result_df.groupby('filename')['type'].nunique() == 4)

369

Only getting those brands who have values for all the four models

In [833]:
count_list = result_df.groupby('filename')['type'].nunique()
valid_brands = count_list[count_list == 4].index.to_list() # having the values of all the four models
len(valid_brands)

369

In [834]:
result_df = result_df[result_df['filename'].isin(valid_brands)]
result_df

,tmp,filename,type,X_Estimate,X_Std. Error,X_t value,X_Pr(>|t|),IV_firststage_reviews_tw_Estimate,IV_firststage_reviews_tw_Std. Error,IV_firststage_reviews_tw_t value,...,HausWutest_Df,HausWutest_F,HausWutest_Pr(>F),Sargan_result_rsq,Sargan_result_adjrsq,Sargan_pvalue_rsq,Sargan_pvalue_adjrsq,num_sig_variables,Y_r.squared,Y_adj.r.squared
0,tmp,Brooks Brothers,fe_reviews_reviews,0.000165,0.000921,0.178737,0.858159,-0.00841370233157332,0.000299929684184101,-28.0522494946145,...,-1,1.79028000162799,0.181016425927441,0.710393089124415,-354.810303025493,0.701035644923341,1,3,0.006867,-0.122601
1,tmp,Brooks Brothers,fe_reviews_visits,-0.000657,0.000817,-0.804520,0.421175,False,False,False,...,-1,5.72760295952137,0.0167752298260142,5.26127891190527e-29,-353.066041581736,1,1,1,0.013417,-0.115198
2,tmp,Brooks Brothers,fe_exp_reviews_reviews,-0.056466,0.037826,-1.492793,0.135620,-0.0370392902690732,0.0203820644439228,-1.81724919823404,...,-1,0.319160504089089,0.572164395901515,1.57075219272356,-353.839371454478,0.455948189079936,1,0,0.014595,-0.113866
3,tmp,Brooks Brothers,fe_exp_reviews_visits,-0.000815,0.041557,-0.019609,0.984357,False,False,False,...,-1,2.2256881669883,0.135861123513664,1.14777777406938e-30,-353.066041581736,1,1,1,0.010238,-0.118791
4,tmp,Torrid,fe_reviews_reviews,0.000178,0.000623,0.285440,0.775330,-0.0010170590792296,0.00188420739611833,-0.539780854976396,...,-1,0.0125725623829476,0.910731020977652,0.326637900513816,-354.102233997887,0.849320254296329,1,1,0.012276,-0.109290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1498,tmp,REI,fe_exp_reviews_visits,-1.019566,1.660166,-0.614135,0.539191,False,False,False,...,-1,0.85244848673874,0.355964534522259,9.51074595213509e-29,-357.421395348837,1,1,0,0.003242,-0.144599
1499,tmp,B,fe_reviews_reviews,-0.068496,0.114223,-0.599671,0.548781,-0.000887747453271405,0.00105785619931258,-0.839194829928951,...,-1,0.511345256595761,0.474624502508825,0.0638736528184297,-356.303645865807,0.968567768050335,1,1,0.000869,-0.128207
1500,tmp,B,fe_reviews_visits,-0.081884,0.171780,-0.476683,0.633630,False,False,False,...,-1,0.332205771928827,0.564415515958776,3.48269244806961e-28,-353.83487716472,1,1,1,0.000485,-0.128641
1501,tmp,B,fe_exp_reviews_reviews,0.418785,0.803571,0.521156,0.602305,-0.00530505854107661,0.000830705541888254,-6.38620819721249,...,-1,0.247612612795232,0.618805337015413,0.0172218787596553,-356.356239582874,0.991426028572975,1,3,0.005295,-0.123209


In [835]:
stat_df = stat_df[stat_df['fileName'].isin(valid_brands)]
stat_df

,tmp,fileName,z_stat,p_value,z_stat_exp,p_value_exp
0,tmp,Enterprise Rent-A-Car,-0.361435,0.717774,-0.927450,0.353693
2,tmp,Cellular Sales,0.722904,0.469739,1.171016,0.241592
3,tmp,lululemon athletica,0.251839,0.801166,-0.245443,0.806113
4,tmp,Dave & Buster's,0.015407,0.987707,0.097575,0.922269
5,tmp,Wetzel's Pretzels,0.588700,0.556063,0.461331,0.644561
...,...,...,...,...,...,...
378,tmp,Pure Hockey,0.307605,0.758383,-0.451526,0.651611
379,tmp,T,-0.607898,0.543255,-0.449707,0.652922
380,tmp,Converse,0.717293,0.473193,-0.347909,0.727909
381,tmp,Dollar Tree,-0.332118,0.739800,-0.369347,0.711869


Extract the significant neighboring brands whose p-values for all the models are significant (<0.05) i.e., X_Pr(>|t|)

In [836]:
def filter_brands_pvalue(brand_pvalue):
    pvalues = brand_pvalue.values
    
    if (pvalues[0] < 0.05) and (pvalues[1] < 0.05) and (pvalues[2] < 0.05) and (pvalues[3] < 0.05):
        return True
    else:
        return False

In [837]:
significant_brands = result_df.groupby('filename')['X_Pr(>|t|)'].apply(filter_brands_pvalue)
significant_brands_list = significant_brands [significant_brands == True].index.to_list()

In [838]:
significant_brands_list

['Cellular Sales', 'Great Clips']

Filter out the results and stats for significant brands only

In [839]:
def get_pvalue_stars(p_value):
    if p_value <= 0.01:
        return '***'
    elif 0.01 < p_value <= 0.05:
        return '**'
    elif 0.05 < p_value <= 0.1:
        return '*'
    else:
        return ''

In [840]:
coeff_est_df = result_df[result_df['filename'].isin(significant_brands_list)]
coeff_est_df['X_pvalue_stars'] = coeff_est_df['X_Pr(>|t|)'].apply(get_pvalue_stars)
coeff_est_df

/tmp/ipykernel_1041426/1034374100.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coeff_est_df['X_pvalue_stars'] = coeff_est_df['X_Pr(>|t|)'].apply(get_pvalue_stars)


,tmp,filename,type,X_Estimate,X_Std. Error,X_t value,X_Pr(>|t|),IV_firststage_reviews_tw_Estimate,IV_firststage_reviews_tw_Std. Error,IV_firststage_reviews_tw_t value,...,HausWutest_F,HausWutest_Pr(>F),Sargan_result_rsq,Sargan_result_adjrsq,Sargan_pvalue_rsq,Sargan_pvalue_adjrsq,num_sig_variables,Y_r.squared,Y_adj.r.squared,X_pvalue_stars
1146,tmp,Cellular Sales,fe_reviews_reviews,0.005808,0.001853,3.133826,0.001949,-2.53140017010446,0.679582061055541,-3.72493671503429,...,1.01636936744029,0.314441183315571,0.690734135579264,-515.886883106838,0.70796043629461,1,3,0.150264,-0.769365,***
1147,tmp,Cellular Sales,fe_reviews_visits,0.004154,0.001342,3.095056,0.002211,False,False,False,...,1.7903314494087,0.182207153779458,3.46059520206023e-18,-508.777777777778,1,1,1,0.154051,-0.761478,***
1148,tmp,Cellular Sales,fe_exp_reviews_reviews,0.106871,0.022461,4.758134,0.000003,-182.45971979629,18.0302998795976,-10.1196164797433,...,5.76892782495933,0.0171064548138775,2.70624672121762,-511.708169513338,0.25843182300593,1,3,0.145416,-0.779458,***
1149,tmp,Cellular Sales,fe_exp_reviews_visits,0.069529,0.022635,3.071751,0.002383,False,False,False,...,0.677865463854577,0.41117557307044,1.26074058169602e-17,-508.777777777778,1,1,1,0.152501,-0.764705,***
1235,tmp,Great Clips,fe_reviews_reviews,-0.454955,0.107815,-4.219756,0.000026,0.0045014982423071,0.00362638778017707,1.2413173976908,...,17.9728633628583,2.39231353712798e-05,1.23192517197869,-380.729066951013,0.540120732349179,1,2,0.001168,-0.230520,***
1236,tmp,Great Clips,fe_reviews_visits,1.659509,0.766009,2.166434,0.030452,False,False,False,...,5.16245867248097,0.0232361194007836,1.7694299329874e-28,-379.227272727273,1,1,1,0.000014,-0.231942,**
1237,tmp,Great Clips,fe_exp_reviews_reviews,-6.586471,2.939338,-2.240801,0.025200,0.012791284226903,0.00887803374194751,1.44077896060092,...,29.5042398330281,6.60472550755767e-08,1.37850308443211,-380.54901933902,0.501951618102956,1,1,0.001160,-0.230530,**
1238,tmp,Great Clips,fe_exp_reviews_visits,11.295771,5.214131,2.166377,0.030456,False,False,False,...,5.2507039043087,0.0220905153953996,1.2297150341538e-30,-379.227272727273,1,1,1,0.000171,-0.231749,**


In [841]:
stat_est_df = stat_df[stat_df['fileName'].isin(significant_brands_list)]
stat_est_df

,tmp,fileName,z_stat,p_value,z_stat_exp,p_value_exp
2,tmp,Cellular Sales,0.722904,0.469739,1.171016,0.241592
293,tmp,Great Clips,-2.733420,0.006268,-2.987566,0.002812


### Making Tables 1 to 3

Renaming the columns to avoid confusion

In [842]:
travel_time = travel_time.rename(columns={'From_PLACEKEY': 'Focal_Stores', 'To_PLACEKEY': 'Neib_Stores'})
travel_time

,Focal_Stores,Neib_Stores,Time_mins
0,zzw-224@62k-p96-s5z,zzw-223@62k-ns4-pn5,20
1,zzw-224@62k-p96-s5z,zzy-222@62k-pd8-975,20
2,zzw-224@62k-p96-s5z,237-222@62k-p8v-z4v,16
3,zzw-224@62k-p96-s5z,222-222@62k-p8v-2p9,12
4,zzw-224@62k-p96-s5z,229-222@62k-p76-d9z,14
...,...,...,...
139637,zzy-22f@62j-shz-vs5,22c-222@62j-sgs-w49,29
139638,zzy-22f@62j-shz-vs5,22k-222@62j-sjr-z9f,21
139639,zzy-22f@62j-shz-vs5,228-222@62j-sj3-v75,14
139640,zzy-22f@62j-shz-vs5,22g-222@62j-sxw-33q,19


1. Extracting all the stores of the significant brands and focal store
2. Dropping all the duplicate rows to have a clean dataframe

In [843]:
brands_visits_focal_sig_neib = brands_visits[(brands_visits['brand_visitation'].isin(significant_brands_list)) | (brands_visits['brand_visitation']==brand)]
brands_visits_focal_sig_neib = brands_visits_focal_sig_neib.drop(columns=['date', 'visits_by_day', 'spend_by_day', 'brand_standard'])
brands_visits_focal_sig_neib = brands_visits_focal_sig_neib.drop_duplicates().reset_index(drop=True)
brands_visits_focal_sig_neib

,PLACEKEY,brand_visitation,lat,lon
0,228-222@62k-rfs-p35,Cellular Sales,42.289695,-71.804286
1,zzw-224@62j-qst-5j9,Great Clips,42.138263,-70.841399
2,zzy-222@62j-sxw-6hq,Victoria's Secret,42.461023,-71.028211
3,zzw-22z@62k-p4d-7wk,Victoria's Secret,41.947084,-71.350412
4,zzw-22j@62j-sbz-wp9,Victoria's Secret,42.482731,-71.213518
5,zzw-224@62k-nzs-nbk,Victoria's Secret,42.092789,-71.267731
6,222-222@62k-37y-m8v,Great Clips,41.618846,-70.490634
7,223-22w@62j-sj3-mhq,Victoria's Secret,42.347431,-71.078613
8,222-222@62j-pm5-jgk,Great Clips,42.623431,-70.655100
9,222-222@62k-qx3-vs5,Great Clips,42.113447,-71.846729


Extracting the time_mins with all the focal stores belonging to brand under consideration, and all of the significant brands

In [844]:
focal_stores = brands_visits_focal_sig_neib[brands_visits_focal_sig_neib['brand_visitation'] == brand]['PLACEKEY'].unique().tolist()
sig_neib_stores = brands_visits_focal_sig_neib[brands_visits_focal_sig_neib['brand_visitation'].isin(significant_brands_list)]['PLACEKEY'].unique().tolist()
travel_time_filtered = travel_time[(travel_time['Focal_Stores'].isin(focal_stores)) & (travel_time['Neib_Stores'].isin(sig_neib_stores))]
travel_time_filtered

,Focal_Stores,Neib_Stores,Time_mins
5350,zzw-223@629-2rt-fpv,zzw-222@629-2rv-j5f,4
5423,zzw-223@629-2rt-fpv,222-222@629-2h4-249,11
5440,zzw-223@629-2rt-fpv,222-222@629-2pw-pvz,19
5471,zzw-223@629-2rt-fpv,zzw-224@629-2px-389,19
5648,zzw-223@629-2rt-fpv,zzw-222@629-2ky-nyv,23
5676,zzw-223@629-2rt-fpv,224-222@629-2ht-vfz,18
25374,zzw-22c@62j-shx-jy9,zzy-223@62j-sry-c89,28
30897,zzw-222@62j-srj-f4v,zzy-223@62j-sry-c89,19
43214,zzy-222@62k-r6j-zpv,223-222@62k-r6t-7kf,16
43300,zzy-222@62k-r6j-zpv,22b-22d@62k-r6k-py9,4


Merging by Neighboring Stores to get the Neib names 

In [845]:
travel_time_filtered = pd.merge(left=travel_time_filtered, right=brands_visits_focal_sig_neib, how='left', left_on='Neib_Stores', right_on='PLACEKEY')
travel_time_filtered = travel_time_filtered.drop(columns=['PLACEKEY','lat', 'lon']).rename(columns={'brand_visitation':'Neib_Brand'})
travel_time_filtered

,Focal_Stores,Neib_Stores,Time_mins,Neib_Brand
0,zzw-223@629-2rt-fpv,zzw-222@629-2rv-j5f,4,Great Clips
1,zzw-223@629-2rt-fpv,222-222@629-2h4-249,11,Great Clips
2,zzw-223@629-2rt-fpv,222-222@629-2pw-pvz,19,Cellular Sales
3,zzw-223@629-2rt-fpv,zzw-224@629-2px-389,19,Great Clips
4,zzw-223@629-2rt-fpv,zzw-222@629-2ky-nyv,23,Great Clips
5,zzw-223@629-2rt-fpv,224-222@629-2ht-vfz,18,Great Clips
6,zzw-22c@62j-shx-jy9,zzy-223@62j-sry-c89,28,Cellular Sales
7,zzw-222@62j-srj-f4v,zzy-223@62j-sry-c89,19,Cellular Sales
8,zzy-222@62k-r6j-zpv,223-222@62k-r6t-7kf,16,Great Clips
9,zzy-222@62k-r6j-zpv,22b-22d@62k-r6k-py9,4,Cellular Sales


Merge by `stat_df`to get z_stat and p-value

In [846]:
travel_time_filtered = pd.merge(left=travel_time_filtered, right=stat_est_df, how='left', left_on='Neib_Brand', right_on='fileName')
travel_time_filtered = travel_time_filtered.drop(columns=['fileName','tmp'])
travel_time_filtered

,Focal_Stores,Neib_Stores,Time_mins,Neib_Brand,z_stat,p_value,z_stat_exp,p_value_exp
0,zzw-223@629-2rt-fpv,zzw-222@629-2rv-j5f,4,Great Clips,-2.733420,0.006268,-2.987566,0.002812
1,zzw-223@629-2rt-fpv,222-222@629-2h4-249,11,Great Clips,-2.733420,0.006268,-2.987566,0.002812
2,zzw-223@629-2rt-fpv,222-222@629-2pw-pvz,19,Cellular Sales,0.722904,0.469739,1.171016,0.241592
3,zzw-223@629-2rt-fpv,zzw-224@629-2px-389,19,Great Clips,-2.733420,0.006268,-2.987566,0.002812
4,zzw-223@629-2rt-fpv,zzw-222@629-2ky-nyv,23,Great Clips,-2.733420,0.006268,-2.987566,0.002812
5,zzw-223@629-2rt-fpv,224-222@629-2ht-vfz,18,Great Clips,-2.733420,0.006268,-2.987566,0.002812
6,zzw-22c@62j-shx-jy9,zzy-223@62j-sry-c89,28,Cellular Sales,0.722904,0.469739,1.171016,0.241592
7,zzw-222@62j-srj-f4v,zzy-223@62j-sry-c89,19,Cellular Sales,0.722904,0.469739,1.171016,0.241592
8,zzy-222@62k-r6j-zpv,223-222@62k-r6t-7kf,16,Great Clips,-2.733420,0.006268,-2.987566,0.002812
9,zzy-222@62k-r6j-zpv,22b-22d@62k-r6k-py9,4,Cellular Sales,0.722904,0.469739,1.171016,0.241592


Calculate linear and exponential coefficients with Instrument Variables

In [847]:
dir_path = os.path.join('iv_results_table_1', brand)
os.makedirs(dir_path, exist_ok=True)

In [848]:
def calc_linear_coeff_iv(group_df):
    neib_brand = group_df.name
    
    # Calculating some metrics about the neighboirng stores
    num_stores = group_df['Neib_Stores'].nunique()
    mean_dist = group_df['Time_mins'].mean()
    median_dist = group_df['Time_mins'].median()
    z_stat = group_df['z_stat'].values[0]
    p_value = group_df['p_value'].values[0]
    
    # Calculating the co-efficient values (2SLS)
    tmp_coeff_est_df = coeff_est_df[(coeff_est_df['filename'] == neib_brand)]

    # Linear Reviews
    x_est_reviews = format(float(tmp_coeff_est_df[tmp_coeff_est_df['type'] == 'fe_reviews_reviews']['X_Estimate'].values[0]), '.2e')
    x_pvalue_star_reviews = tmp_coeff_est_df[tmp_coeff_est_df['type'] == 'fe_reviews_reviews']['X_pvalue_stars'].values[0]
    x_std_err_reviews = format(float(tmp_coeff_est_df[tmp_coeff_est_df['type'] == 'fe_reviews_reviews']['X_Std. Error'].values[0]), '.2e')
    reviews = str(x_est_reviews) + x_pvalue_star_reviews + "({})".format(x_std_err_reviews)
    
    # Linear Visits
    x_est_visits = format(float(tmp_coeff_est_df[tmp_coeff_est_df['type'] == 'fe_reviews_visits']['X_Estimate'].values[0]), '.2e')
    x_pvalue_star_visits = tmp_coeff_est_df[tmp_coeff_est_df['type'] == 'fe_reviews_visits']['X_pvalue_stars'].values[0]
    x_std_err_visits = format(float(tmp_coeff_est_df[tmp_coeff_est_df['type'] == 'fe_reviews_visits']['X_Std. Error'].values[0]), '.2e')
    visits = str(x_est_visits) + x_pvalue_star_visits + "({})".format(x_std_err_visits)
    
    # Calculating the co-efficient values (OLS)
    tmp_full_result_df = full_result_df[(full_result_df['filename'] == neib_brand)]
    
    x_est_ols = format(float(tmp_full_result_df[tmp_full_result_df['type'] == 'ols']['X_Estimate'].values[0]), '.2e')
    x_std_err_ols = format(float(tmp_full_result_df[tmp_full_result_df['type'] == 'ols']['X_Std. Error'].values[0]), '.2e')
    ols = str(x_est_ols)  + "({})".format(x_std_err_ols)
    
    return pd.Series([neib_brand, reviews, visits, ols, num_stores, mean_dist, median_dist, z_stat, p_value],
                     index=['Neighboring brand', '2SLS (Linear Reviews)', '2SLS (Linear Visits)', 'OLS', 'Num of Stores', 'Mean Dist',
                            'Median Dist', 'z-stat', 'p-value'])

In [849]:
def calc_exp_coeff_iv(group_df):
    neib_brand = group_df.name
    
    # Calculating some metrics about the neighboirng stores
    num_stores = group_df['Neib_Stores'].nunique()
    mean_dist = group_df['Time_mins'].mean()
    median_dist = group_df['Time_mins'].median()
    z_stat = group_df['z_stat_exp'].values[0]
    p_value = group_df['p_value_exp'].values[0]
    
    # Calculating the co-efficient values (2SLS)
    tmp_coeff_est_df = coeff_est_df[(coeff_est_df['filename'] == neib_brand)]

    # Linear Reviews
    x_est_reviews = format(float(tmp_coeff_est_df[tmp_coeff_est_df['type'] == 'fe_exp_reviews_reviews']['X_Estimate'].values[0]), '.2e')
    x_pvalue_star_reviews = tmp_coeff_est_df[tmp_coeff_est_df['type'] == 'fe_exp_reviews_reviews']['X_pvalue_stars'].values[0]
    x_std_err_reviews = format(float(tmp_coeff_est_df[tmp_coeff_est_df['type'] == 'fe_exp_reviews_reviews']['X_Std. Error'].values[0]), '.2e')
    reviews = str(x_est_reviews) + x_pvalue_star_reviews + "({})".format(x_std_err_reviews)
    
    # Linear Visits
    x_est_visits = format(float(tmp_coeff_est_df[tmp_coeff_est_df['type'] == 'fe_exp_reviews_visits']['X_Estimate'].values[0]), '.2e')
    x_pvalue_star_visits = tmp_coeff_est_df[tmp_coeff_est_df['type'] == 'fe_exp_reviews_visits']['X_pvalue_stars'].values[0]
    x_std_err_visits = format(float(tmp_coeff_est_df[tmp_coeff_est_df['type'] == 'fe_exp_reviews_visits']['X_Std. Error'].values[0]), '.2e')
    visits = str(x_est_visits) + x_pvalue_star_visits + "({})".format(x_std_err_visits)
    
    # Calculating the co-efficient values (OLS)
    tmp_full_result_df = full_result_df[(full_result_df['filename'] == neib_brand)]
    
    x_est_ols = format(float(tmp_full_result_df[tmp_full_result_df['type'] == 'm_olsExp']['X_Estimate'].values[0]), '.2e')
    x_std_err_ols = format(float(tmp_full_result_df[tmp_full_result_df['type'] == 'm_olsExp']['X_Std. Error'].values[0]), '.2e')
    ols = str(x_est_ols)  + "({})".format(x_std_err_ols)
    
    return pd.Series([neib_brand, reviews, visits, ols, num_stores, mean_dist, median_dist, z_stat, p_value],
                     index=['Neighboring brand', '2SLS (Exp Reviews)', '2SLS (Exp Visits)', 'OLS_Exp', 'Num of Stores', 'Mean Dist',
                            'Median Dist', 'z-stat_exp', 'p-value_exp'])

In [850]:
linear_coeff_iv = travel_time_filtered.groupby('Neib_Brand')[['Neib_Stores', 'Time_mins', 'z_stat', 'p_value']].apply(calc_linear_coeff_iv).reset_index(drop=True)
linear_coeff_iv.to_latex(os.path.join(dir_path, 'linear_coeff_iv.tex'))
linear_coeff_iv.head()

,Neighboring brand,2SLS (Linear Reviews),2SLS (Linear Visits),OLS,Num of Stores,Mean Dist,Median Dist,z-stat,p-value
0,Cellular Sales,5.81e-03***(1.85e-03),4.15e-03***(1.34e-03),3.75e-03(1.49e-03),3,18.833333,19.0,0.722904,0.469739
1,Great Clips,-4.55e-01***(1.08e-01),1.66e+00**(7.66e-01),-5.18e-03(2.97e-02),11,14.250000,14.0,-2.733420,0.006268


In [851]:
exp_coeff_iv = travel_time_filtered.groupby('Neib_Brand')[['Neib_Stores', 'Time_mins', 'z_stat_exp', 'p_value_exp']].apply(calc_exp_coeff_iv).reset_index(drop=True)
exp_coeff_iv.to_latex(os.path.join(dir_path, 'exp_coeff_iv.tex'))
exp_coeff_iv.head()

,Neighboring brand,2SLS (Exp Reviews),2SLS (Exp Visits),OLS_Exp,Num of Stores,Mean Dist,Median Dist,z-stat_exp,p-value_exp
0,Cellular Sales,1.07e-01***(2.25e-02),6.95e-02***(2.26e-02),5.98e-02(2.32e-02),3,18.833333,19.0,1.171016,0.241592
1,Great Clips,-6.59e+00**(2.94e+00),1.13e+01**(5.21e+00),-1.23e-01(2.82e-01),11,14.250000,14.0,-2.987566,0.002812


### Making the Tables E1 to E6

1. Put stars at the neighboring brands as per the p-value for each model \
-> *** means p_value <= 0.01 \
-> ** means 0.01 < p_value <= 0.05 \
-> * means 0.05 < p_value <= 0.1

Firstly creating the directory to store the results

In [ ]:
dir_path = os.path.join('iv_results_table_e', brand)
os.makedirs(dir_path, exist_ok=True)

In [ ]:
lin_reviews = coeff_est_df[coeff_est_df['type'] == 'fe_reviews_reviews'].sort_values('X_Estimate', ascending=False)
lin_visits = coeff_est_df[coeff_est_df['type'] == 'fe_reviews_visits'].sort_values('X_Estimate', ascending=False)
exp_reviews = coeff_est_df[coeff_est_df['type'] == 'fe_exp_reviews_reviews'].sort_values('X_Estimate', ascending=False)
exp_visits = coeff_est_df[coeff_est_df['type'] == 'fe_exp_reviews_visits'].sort_values('X_Estimate', ascending=False)

In [ ]:
def prepare_first_stage_results_review(row):
    
    neib_brand = row['filename']
    
    # Twitter Vaues
    tw_estimate = format(float(row['IV_firststage_reviews_tw_Estimate']), '.2e')
    tw_std_err = format(float(row['IV_firststage_reviews_tw_Std. Error']), '.2e')
    tw_p_value = float(row['IV_firststage_reviews_tw_Pr(>|t|)'])
    
    # Facebook Vaues
    fb_estimate = format(float(row['IV_firststage_reviews_fb_Estimate']), '.2e')
    fb_std_err = format(float(row['IV_firststage_reviews_fb_Std. Error']), '.2e')
    fb_p_value = float(row['IV_firststage_reviews_fb_Pr(>|t|)'])
    
    # Instagram Vaues
    ig_estimate = format(float(row['IV_firststage_reviews_ig_Estimate']), '.2e')
    ig_std_err = format(float(row['IV_firststage_reviews_ig_Std. Error']), '.2e')
    ig_p_value = float(row['IV_firststage_reviews_ig_Pr(>|t|)'])
    
    # Getting table entries
    twitter_likes = str(tw_estimate) + get_pvalue_stars(tw_p_value) + "({})".format(tw_std_err)
    facebook_likes = str(fb_estimate) + get_pvalue_stars(fb_p_value) + "({})".format(fb_std_err)
    instagram_likes = str(ig_estimate) + get_pvalue_stars(ig_p_value) + "({})".format(ig_std_err)
    
    # Calculating significant variables
    num_sig = 0.0
    
    if (tw_p_value <= 0.05):
        num_sig += 1

    if (fb_p_value <= 0.05):
        num_sig += 1

    if (ig_p_value <= 0.05):
        num_sig += 1
    
    # Getting WaldTest F-Stats
    wald_f_stat = format(float(row['WaldTest_F']), '.4')
        
    return pd.Series([neib_brand, twitter_likes, facebook_likes, instagram_likes, num_sig, wald_f_stat],
                     index=['Neighboring brand', 'Twitter Likes', 'Facebook Likes', 'Instagram Likes', 'Num Sig', 'Weka Instrument (F-stats)'])

In [ ]:
def prepare_first_stage_results_visits(row):
    
    neib_brand = row['filename']
    
    # Visits Vaues
    visits_estimate = format(float(row['IV_firststage_visits_Estimate']), '.2e')
    visits_std_err = format(float(row['IV_firststage_visits_Std. Error']), '.2e')
    visits_p_value = float(row['IV_firststage_visits_Pr(>|t|)'])
        
    # Getting table entries
    visits = str(visits_estimate) + get_pvalue_stars(visits_p_value) + "({})".format(visits_std_err)
        
    # Getting WaldTest F-Stats
    wald_f_stat = format(float(row['WaldTest_F']), '.4')
        
    return pd.Series([neib_brand, visits, wald_f_stat],
                     index=['Neighboring brand', 'Visits', 'Weka Instrument (F-stats)'])

In [ ]:
def prepare_instrumental_validity_visits(row):
        
    neib_brand = row['filename']
    # Getting Wu Hausman and Sargan p-value
    hausman_pvalue = format(float(row['HausWutest_Pr(>F)']),'.4')
    sargan_pvalue = format(float(row['Sargan_pvalue_rsq']), '.4' )
    
    return pd.Series([neib_brand, hausman_pvalue, sargan_pvalue],
                     index=['Neighboring brand', 'Wu Hausman (p-value)', 'Sargan (p-value)'])

In [ ]:
first_stage_linear_reviews = lin_reviews.apply(prepare_first_stage_results_review, axis=1, result_type='expand').reset_index(drop=True)
first_stage_linear_reviews.to_latex(os.path.join(dir_path, 'first_stage_reviews.tex'))
first_stage_linear_reviews.head()

In [ ]:
first_stage_linear_visits = lin_visits.apply(prepare_first_stage_results_visits, axis=1, result_type='expand').reset_index(drop=True)
first_stage_linear_visits.to_latex(os.path.join(dir_path, 'first_stage_visits.tex'))
first_stage_linear_visits.head()

In [ ]:
first_stage_exp_reviews = exp_reviews.apply(prepare_first_stage_results_review, axis=1, result_type='expand').reset_index(drop=True)
first_stage_exp_reviews.to_latex(os.path.join(dir_path, 'first_stage_reviews_exp.tex'))
first_stage_exp_reviews.head()

In [ ]:
first_stage_exp_visits = exp_visits.apply(prepare_first_stage_results_visits, axis=1, result_type='expand').reset_index(drop=True)
first_stage_exp_visits.to_latex(os.path.join(dir_path, 'first_stage_visits_exp.tex'))
first_stage_exp_visits.head()

In [ ]:
instrumental_validity_lin_reviews = lin_reviews.apply(prepare_instrumental_validity_visits, axis=1, result_type='expand').reset_index(drop=True)
instrumental_validity_lin_reviews.to_latex(os.path.join(dir_path, 'iv_valid_reviews.tex'))
instrumental_validity_lin_reviews.head()

In [ ]:
instrumental_validity_lin_visits = lin_visits.apply(prepare_instrumental_validity_visits, axis=1, result_type='expand').reset_index(drop=True)
instrumental_validity_lin_visits.to_latex(os.path.join(dir_path, 'iv_valid_visits.tex'))
instrumental_validity_lin_visits.head()

In [ ]:
instrumental_validity_exp_reviews = exp_reviews.apply(prepare_instrumental_validity_visits, axis=1, result_type='expand').reset_index(drop=True)
instrumental_validity_exp_reviews.to_latex(os.path.join(dir_path, 'iv_valid_reviews_exp.tex'))
instrumental_validity_exp_reviews.head()

In [ ]:
instrumental_validity_exp_visits = exp_visits.apply(prepare_instrumental_validity_visits, axis=1, result_type='expand').reset_index(drop=True)
instrumental_validity_exp_visits.to_latex(os.path.join(dir_path, 'iv_valid_visits_exp.tex'))
instrumental_validity_exp_visits.head()

### Making the Figure D1 and D2

1. Plot their X_Estimate for each of the four models
2. Put stars at the neighboring brands as per the p-value for each model \
-> *** means p_value <= 0.01 \
-> ** means 0.01 < p_value <= 0.05 \
-> * means 0.05 < p_value <= 0.1

In [ ]:
dir_path = os.path.join('est_coeff_2SLS', brand)
os.makedirs(dir_path, exist_ok=True)

In [ ]:
custom_colors = {
    'positive': '#6488ea',  # Blue for positive estimates
    'star': '#922b05',  # Red for negative estimates
    'negative': '#3d9973'       # Prettier yellow for stars
}

confidence_level = 0.95
lin_reviews['Lower Bound'] = lin_reviews['X_Estimate'] - 1.96 * lin_reviews['X_Std. Error']
lin_reviews['Upper Bound'] = lin_reviews['X_Estimate'] + 1.96 * lin_reviews['X_Std. Error']

# Create figure and axes
fig, ax = plt.subplots(figsize=(8,8))

# Add a constant offset for stars
star_offset = 0.1
    
# Set the font size for the stars
star_fontsize = 15

# Plotting estimates and confidence intervals for each business
for idx, row in lin_reviews.iterrows():
    biz_biz = row['filename']
    estimate = row['X_Estimate']
    std_error = row['X_Std. Error']
    lower_bound = row['Lower Bound']
    upper_bound = row['Upper Bound']

    # Determine the color based on the sign of the estimate
    color = custom_colors['positive'] if estimate >= 0 else custom_colors['negative']

    # Plotting the bars with different colors for positive and negative estimates
    ax.barh(biz_biz, estimate, xerr=[[estimate - lower_bound], [upper_bound - estimate]], color=color)

    # Add stars based on the number of *s in the "coefficients" column with an offset
    stars = row['X_pvalue_stars'].count('*')
    if stars > 0:
        # Set the alignment for negative estimates
        ha = 'left' if estimate >= 0 else 'right'
        ax.text(estimate + star_offset if estimate >= 0 else estimate - star_offset, biz_biz, '*' * stars, va='center', ha=ha, color=custom_colors['star'], fontsize=star_fontsize)

plt.axvline(x=0, linestyle='--', color='gray')  # Plotting a dashed line for the estimate

plt.tight_layout()

# Add labels and title
ax.set_xscale('symlog')
ax.set_xmargin(0.05)
plt.savefig(os.path.join(dir_path, 'reviews.png'))
# Show the plot
plt.show()


In [ ]:
custom_colors = {
    'positive': '#6488ea',  # Blue for positive estimates
    'star': '#922b05',  # Red for negative estimates
    'negative': '#3d9973'       # Prettier yellow for stars
}

confidence_level = 0.95
lin_visits['Lower Bound'] = lin_visits['X_Estimate'] - 1.96 * lin_visits['X_Std. Error']
lin_visits['Upper Bound'] = lin_visits['X_Estimate'] + 1.96 * lin_visits['X_Std. Error']

# Create figure and axes
fig, ax = plt.subplots(figsize=(8,8))

# Add a constant offset for stars
star_offset = 0.1
    
# Set the font size for the stars
star_fontsize = 15

# Plotting estimates and confidence intervals for each business
for idx, row in lin_visits.iterrows():
    biz_biz = row['filename']
    estimate = row['X_Estimate']
    std_error = row['X_Std. Error']
    lower_bound = row['Lower Bound']
    upper_bound = row['Upper Bound']

    # Determine the color based on the sign of the estimate
    color = custom_colors['positive'] if estimate >= 0 else custom_colors['negative']

    # Plotting the bars with different colors for positive and negative estimates
    ax.barh(biz_biz, estimate, xerr=[[estimate - lower_bound], [upper_bound - estimate]], color=color)

    # Add stars based on the number of *s in the "coefficients" column with an offset
    stars = row['X_pvalue_stars'].count('*')
    if stars > 0:
        # Set the alignment for negative estimates
        ha = 'left' if estimate >= 0 else 'right'
        ax.text(estimate + star_offset if estimate >= 0 else estimate - star_offset, biz_biz, '*' * stars, va='center', ha=ha, color=custom_colors['star'], fontsize=star_fontsize)

plt.axvline(x=0, linestyle='--', color='gray')  # Plotting a dashed line for the estimate

plt.tight_layout()

# Add labels and title
ax.set_xscale('symlog')
ax.set_xmargin(0.05)
plt.savefig(os.path.join(dir_path, 'visits.png'))
# Show the plot
plt.show()


In [ ]:
custom_colors = {
    'positive': '#6488ea',  # Blue for positive estimates
    'star': '#922b05',  # Red for negative estimates
    'negative': '#3d9973'       # Prettier yellow for stars
}

confidence_level = 0.95
exp_reviews['Lower Bound'] = exp_reviews['X_Estimate'] - 1.96 * exp_reviews['X_Std. Error']
exp_reviews['Upper Bound'] = exp_reviews['X_Estimate'] + 1.96 * exp_reviews['X_Std. Error']

# Create figure and axes
fig, ax = plt.subplots(figsize=(8,8))

# Add a constant offset for stars
star_offset = 0.1
    
# Set the font size for the stars
star_fontsize = 15

# Plotting estimates and confidence intervals for each business
for idx, row in exp_reviews.iterrows():
    biz_biz = row['filename']
    estimate = row['X_Estimate']
    std_error = row['X_Std. Error']
    lower_bound = row['Lower Bound']
    upper_bound = row['Upper Bound']

    # Determine the color based on the sign of the estimate
    color = custom_colors['positive'] if estimate >= 0 else custom_colors['negative']

    # Plotting the bars with different colors for positive and negative estimates
    ax.barh(biz_biz, estimate, xerr=[[estimate - lower_bound], [upper_bound - estimate]], color=color)

    # Add stars based on the number of *s in the "coefficients" column with an offset
    stars = row['X_pvalue_stars'].count('*')
    if stars > 0:
        # Set the alignment for negative estimates
        ha = 'left' if estimate >= 0 else 'right'
        ax.text(estimate + star_offset if estimate >= 0 else estimate - star_offset, biz_biz, '*' * stars, va='center', ha=ha, color=custom_colors['star'], fontsize=star_fontsize)

plt.axvline(x=0, linestyle='--', color='gray')  # Plotting a dashed line for the estimate

plt.tight_layout()

# Add labels and title
ax.set_xscale('symlog')
ax.set_xmargin(0.05)

plt.savefig(os.path.join(dir_path, 'reviews_exp.png'))
# Show the plot
plt.show()


In [ ]:
custom_colors = {
    'positive': '#6488ea',  # Blue for positive estimates
    'star': '#922b05',  # Red for negative estimates
    'negative': '#3d9973'       # Prettier yellow for stars
}

confidence_level = 0.95
exp_visits['Lower Bound'] = exp_visits['X_Estimate'] - 1.96 * exp_visits['X_Std. Error']
exp_visits['Upper Bound'] = exp_visits['X_Estimate'] + 1.96 * exp_visits['X_Std. Error']

# Create figure and axes
fig, ax = plt.subplots(figsize=(8,8))

# Add a constant offset for stars
star_offset = 0.1
    
# Set the font size for the stars
star_fontsize = 15

# Plotting estimates and confidence intervals for each business
for idx, row in exp_visits.iterrows():
    biz_biz = row['filename']
    estimate = row['X_Estimate']
    std_error = row['X_Std. Error']
    lower_bound = row['Lower Bound']
    upper_bound = row['Upper Bound']

    # Determine the color based on the sign of the estimate
    color = custom_colors['positive'] if estimate >= 0 else custom_colors['negative']

    # Plotting the bars with different colors for positive and negative estimates
    ax.barh(biz_biz, estimate, xerr=[[estimate - lower_bound], [upper_bound - estimate]], color=color)

    # Add stars based on the number of *s in the "coefficients" column with an offset
    stars = row['X_pvalue_stars'].count('*')
    if stars > 0:
        # Set the alignment for negative estimates
        ha = 'left' if estimate >= 0 else 'right'
        ax.text(estimate + star_offset if estimate >= 0 else estimate - star_offset, biz_biz, '*' * stars, va='center', ha=ha, color=custom_colors['star'], fontsize=star_fontsize)

plt.axvline(x=0, linestyle='--', color='gray')  # Plotting a dashed line for the estimate

plt.tight_layout()

# Add labels and title
ax.set_xscale('symlog')
ax.set_xmargin(0.05)
plt.savefig(os.path.join(dir_path, 'visits_exp.png'))
# Show the plot
plt.show()